# Group 7
## Questions
1. How many of the world's 1-year old children today have been vaccinated against some disease?
2. How many against more diseases?
3. How has the rate of vaccination for different diseases changed over time?
4. Are there country characteristics that predict vaccination levels, or trends in vaccination levels?

## Datasets
1. The data for the immunization coverage among 1-year-olds is provided by thw Wolrd Health Organization (https://www.who.int/data/gho/gho-search?indexCatalogue=ghosearchindex&searchQuery=immunization%20coverage%20among%201-year-olds&wordsMode=AllWords)
2. ... to be continued for country characteristics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import typing
import json

In [2]:
# Mount google drive if notebook is running in colab
try:
  from google.colab import drive
  drive.mount("/content/drive", force_remount=True)
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
  base_data_path = os.path.join(os.curdir, 'drive', 'MyDrive', 'DOPP', 'data')
else:
  base_data_path = os.path.join(os.curdir, 'data')

## Helpers Functions

In [4]:
mapping = [
    ('WHS4_100', "diphtheria_pertussis_tetanus"),
    ('WHS4_129', "haemophilus_influenzae"),
    ('WHS4_117', "hepatitisB"),
    ('WHS8_110', "measles"),
    ('WHS4_544', "polio"),
    ('ROTAC', "rotavirus"),
    ('PCV3', "streptococcus_pneumoniae"),
    ('WHS4_543', "tuberculosis"),
]


def diseaseNameToVaccineIndicator(disease: str) -> str:
    for e in mapping:
        if e[1] == disease:
            return e[0]

    raise ValueError("Invalid disease")


def vaccineIndicatorToDiseaseName(indicator: str) -> str:
    for e in mapping:
        if e[0] == indicator:
            return e[1]

    raise ValueError("Invalid indicator")


def printDataDimensions(df: pd.DataFrame) -> None:
    print(f"Data dimensions are: {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"\nindex types are: \n-----------------\n{df.index.dtypes}")
    print(f"\ncolumn types are: \n-----------------\n{df.dtypes}")

    print(
        f"\nFor daily data: \nData dimensions are: {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"\nindex types are: \n-----------------\n{df.index.dtypes}")
    print(f"\ncolumn types are: \n-----------------\n{df.dtypes}")


# Load JSON country code mapping
country_mapping_path = os.path.join(base_data_path, 'country_code_mapping.json')
country_code_list = []
with open(country_mapping_path) as f:
    data = json.load(f)

    for item in data:
        country_data = {'alpha2': None, 'alpha3': None, 'country': None}
        country_data['alpha2'] = item['alpha2']
        country_data['alpha3'] = item['alpha3']
        country_data['country'] = item['country']
        country_code_list.append(country_data)

def getCountryMapping(search: str):
    searchLc = search.lower()
    for item in country_code_list:
        if item['country'] == searchLc or item['alpha2'] == searchLc or item['alpha3'] == searchLc:
            return item

    raise ValueError(f'Invalid country {search}')



## 1. Load Data
### 1.1 Load Immunization Data

In [5]:
def load_immunization_data(immunization_data_path: str) -> pd.DataFrame:
  immunization_data_directory = glob.glob(os.path.join(immunization_data_path, '*.csv'))

  immunization_data_list = []
  for filename in immunization_data_directory:
      df = pd.read_csv(filename, sep=',') 
      indicator = df['IndicatorCode'][0]
      df.rename(columns={'Period': 'year', 'Location': 'country', 'Value': indicator, 'SpatialDimValueCode': 'alpha3' }, inplace=True)
      df = df.assign(alpha2=lambda x: x.alpha3.map(lambda y: getCountryMapping(y).get('alpha2')))
      df = df[['year', 'country', 'alpha2', 'alpha3', indicator]].copy()
      df['country'] = df['country'].str.lower()
      df['alpha3'] = df['alpha3'].str.lower()
      df.set_index(['year', 'country', 'alpha2', 'alpha3'], inplace=True)
      immunization_data_list.append(df)

  immunization_data = pd.concat(immunization_data_list, axis=1)
  
  immunization_data[diseaseNameToVaccineIndicator('diphtheria_pertussis_tetanus')] = immunization_data[diseaseNameToVaccineIndicator('diphtheria_pertussis_tetanus')].astype(float)
  immunization_data[diseaseNameToVaccineIndicator('measles')] = immunization_data[diseaseNameToVaccineIndicator('measles')].astype(float)

  immunization_data.sort_values(['year'], inplace=True)

  return immunization_data

In [6]:
immunization_data_path = os.path.join(base_data_path, 'immunization')
immunization_data = load_immunization_data(immunization_data_path)

display(immunization_data[immunization_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(immunization_data)

,,,,WHS4_100,WHS4_129,WHS4_117,WHS8_110,WHS4_544,ROTAC,PCV3,WHS4_543
year,country,alpha2,alpha3,,,,,,,,
2000,austria,at,aut,81.0,72.0,33.0,75.0,71.0,NaN,NaN,NaN
2001,austria,at,aut,84.0,74.0,44.0,79.0,83.0,NaN,NaN,NaN
2002,austria,at,aut,83.0,82.0,81.0,78.0,82.0,NaN,NaN,NaN
2003,austria,at,aut,84.0,84.0,83.0,79.0,84.0,NaN,NaN,NaN
2004,austria,at,aut,83.0,83.0,83.0,74.0,83.0,NaN,NaN,NaN
2005,austria,at,aut,86.0,86.0,86.0,75.0,86.0,NaN,NaN,NaN
2006,austria,at,aut,83.0,83.0,83.0,80.0,83.0,NaN,NaN,NaN
2007,austria,at,aut,85.0,85.0,85.0,79.0,85.0,NaN,NaN,NaN
2008,austria,at,aut,83.0,83.0,83.0,83.0,83.0,47.0,NaN,NaN


### 1.2 Load Infant Mortality Data

In [7]:
def load_infant_mortality_data(infant_mortality_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(infant_mortality_data_path, sep=',') 
    df.rename(columns={'Period': 'year', 'Location': 'country', 'FactValueNumeric': 'infant_mortality', 'SpatialDimValueCode': 'alpha3'}, inplace=True)
    df = df.assign(alpha2=lambda x: x.alpha3.map(lambda y: getCountryMapping(y).get('alpha2')))
    df = df.loc[df['Dim1'] == 'Both sexes']
    df = df[['year', 'country', 'alpha2', 'alpha3', 'infant_mortality']].copy()
    df['country'] = df['country'].str.lower()
    df['alpha3'] = df['alpha3'].str.lower()
    df.set_index(['year', 'country', 'alpha2', 'alpha3'], inplace=True)
    df.sort_values(['year'], inplace=True)

    return df


In [8]:
infant_mortality_data_path = os.path.join(base_data_path, 'country_characteristics', 'infant_mortality_rate.csv')
infant_mortality_data = load_infant_mortality_data(infant_mortality_data_path)

display(infant_mortality_data[infant_mortality_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(infant_mortality_data)

,,,,infant_mortality
year,country,alpha2,alpha3,
1949,austria,at,aut,83.37
1950,austria,at,aut,73.12
1951,austria,at,aut,64.59
1952,austria,at,aut,57.84
1953,austria,at,aut,52.72
...,...,...,...,...
2016,austria,at,aut,2.97
2017,austria,at,aut,2.93
2018,austria,at,aut,2.92


### 1.3 Load Education Data

In [9]:
def load_education_data(education_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(education_data_path, sep=',') 
    df.rename(columns={'TIME_PERIOD': 'year', 'geo': 'alpha2','OBS_VALUE': 'education_level'}, inplace=True) 
    df = df.loc[(df['sex'] == 'T') & (df['age'] == 'Y15-74') & (df['isced11'] == 'ED3-8') & (df['alpha2'] != 'EA19') & (df['alpha2'] != 'EU15') & (df['alpha2'] != 'EU28') & (df['alpha2'] != 'EU27_2020')]
    df['alpha2'] = df['alpha2'].str.lower() 
    df = df.assign(country=lambda x: x.alpha2.map(lambda y: getCountryMapping(y).get('country')))
    df = df.assign(alpha3=lambda x: x.alpha2.map(lambda y: getCountryMapping(y).get('alpha3')))
    df = df[['year', 'country', 'alpha2', 'alpha3', 'education_level']].copy()
    df['country'] = df['country'].str.lower()
    df.set_index(['year', 'country', 'alpha2', 'alpha3'], inplace=True)
    df.sort_values(['year'], inplace=True)

    return df

In [10]:
education_data_path = os.path.join(base_data_path, 'country_characteristics', 'education.csv')
education_data = load_education_data(education_data_path)

display(education_data[education_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(education_data)

,,,,education_level
year,country,alpha2,alpha3,
2004,austria,at,aut,72.5
2005,austria,at,aut,73.2
2006,austria,at,aut,72.7
2007,austria,at,aut,72.4
2008,austria,at,aut,73.5
2009,austria,at,aut,74.5
2010,austria,at,aut,75.2
2011,austria,at,aut,75.3
2012,austria,at,aut,76.0


#To do
1. How many of the world's 1-year old children today have been vaccinated against some disease?
2. How many against more diseases?
3. How has the rate of vaccination for different diseases changed over time?

Zunächst reicht nur Österreich, wenn Zeit ist, können wir noch andere Länder hinzunehmen. Mit Seaborn kann man besonders schöne Plots erstellen, wer mag, muss aber nicht sein, matplotlib reicht auch
1. Plotten der Prozentzahl jeder Krankheit aktuell (also letzte Jahreszahl die wir haben, in dem Falle 2021) von Österreich
2. Wird mit 1 eigentlich beantwortet, wüsste nicht, was da noch mehr hingehört
3. Plotten der Prozentzahl jeder Krankheit über die Zeit von Österreich

Wenn fertig, kann man ja noch andere Länder nehmen. Frankreich oder Italieren würde sich womöglich anbieten

## Are there country characteristics that predict vaccination levels, or trends in vaccination levels?
Testing correlations between certain country characteristics and vaccination levels. Can we achieve accurate predictions? We will use logistic regression. Used country characteristics (Austria): education level, infant mortality rate, (gdp per capita), vaccine mandates

The number of diseases we are looking at is 8

## TO DO
Umschreiben zu Funktionen.. Umsetzung für mindestens 3 Länder: Österreich, Frankreich, Italien, Deutschland, Spanien? Damit Vorhersage und Korrelation aussagekräftiger ist für mehr Daten

In [11]:
mandates_data_path = os.path.join(base_data_path, 'country_characteristics')

In [12]:
characteristics_list = []

#First the mortality rates for Austria between 2014 and 2020
mortality_at_data = infant_mortality_data[(infant_mortality_data.index.get_level_values('country') == 'austria') & (infant_mortality_data.index.get_level_values('year') >= 2014)].copy()
mortality_at_data = mortality_at_data.droplevel(1)
mortality_at_data = mortality_at_data.loc[mortality_at_data.index.repeat(8)]
mortality_at_data = mortality_at_data.reset_index()
mortality_at_data.drop(columns=['alpha2', 'alpha3'], inplace=True)
characteristics_list.append(mortality_at_data)

#Now the education level for Austria, highest education level achieved in group 18-64, between 2014 and 2020
education_at_data = education_data[(education_data.index.get_level_values('country') == 'austria') & (education_data.index.get_level_values('year') >= 2014) & (education_data.index.get_level_values('year') < 2021)].copy()
education_at_data = education_at_data.droplevel(1)
education_at_data = education_at_data.loc[education_at_data.index.repeat(8)]
education_at_data = education_at_data.reset_index()
education_at_data.drop(columns=['alpha2', 'alpha3'], inplace=True)
characteristics_list.append(education_at_data)

#And last the vaccine mandates for Austria between the times 2014 and 2020
df = pd.read_csv(os.path.join(mandates_data_path, "mandates.csv"), sep=',')
df['country'] = df['country'].str.lower()
df.set_index(['year', 'country'], inplace=True)

mandates_at_data = df[(df.index.get_level_values('country') == 'austria') & (df.index.get_level_values('year') >= 2014) & (df.index.get_level_values('year') < 2021)].copy()
mandates_at_data = mandates_at_data.droplevel(1)
mandates_at_data = mandates_at_data.reset_index()
mandates_at_data.rename(columns={'year': 'year_m'}, inplace=True) 
characteristics_list.append(mandates_at_data)


In [13]:
#Data for building our prediction and correlation model
immunization_at_data = immunization_data[(immunization_data.index.get_level_values('country') == 'austria')].reset_index()
immunization_at_data = immunization_at_data.loc[(immunization_at_data['year'] >= 2014) & (immunization_at_data['year'] < 2021)]
immunization_at_data = immunization_at_data.drop(labels='country', axis=1)
immunization_at_data = immunization_at_data.sort_values(by="year")
cols = immunization_at_data.columns[1:]

characteristics_data = pd.concat(characteristics_list, axis=1, ignore_index=False)
characteristics_data = characteristics_data.drop(labels=['year'], axis=1)
characteristics_data = characteristics_data.sort_values(by='year_m')

for el in cols:
    df_el = immunization_at_data[['year', el]].dropna()
    for index, row in df_el.iterrows():
        characteristics_data.loc[(characteristics_data['IndicatorCode'] == el.upper()) & (characteristics_data['year_m'] == int(row['year'])), 'immunization'] = row[el]

display(characteristics_data.head())


,infant_mortality,education_level,year_m,IndicatorCode,recommended,mandatory,funded,immunization
55,2.97,80.3,2014,ROTAC,1,0,1,61.0
53,2.97,80.3,2014,PCV3,1,0,1,NaN
52,2.97,80.3,2014,WHS8_110,1,0,1,96.0
54,2.97,80.3,2014,WHS4_544,1,0,1,98.0
6,3.10,78.3,2014,WHS4_543,0,0,0,NaN


## 2. Merge Data

In [14]:
def merge_data(immunization_data: pd.DataFrame, infant_mortality_data: pd.DataFrame, education_data: pd.DataFrame) -> pd.DataFrame:
    merged_data = pd.merge(immunization_data, infant_mortality_data, how='inner', left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, education_data, how='inner', left_index=True, right_index=True)

    return merged_data

In [15]:
data_merged = merge_data(immunization_data, infant_mortality_data, education_data)
display(data_merged)
# printDataDimensions(data_merged)


WHS4_100  WHS4_129  WHS4_117  WHS8_110  \
year country     alpha2 alpha3                                           
2004 austria     at     aut         83.0      83.0      83.0      74.0   
     malta       mt     mlt         89.0      83.0      85.0      94.0   
     ireland     ie     irl         89.0      89.0       NaN      81.0   
     netherlands nl     nld         98.0      97.0       NaN      96.0   
     france      fr     fra         98.0      97.0      35.0      88.0   
...                                  ...       ...       ...       ...   
2020 france      fr     fra         96.0      95.0      91.0      92.0   
     cyprus      cy     cyp         96.0      92.0      94.0      86.0   
     slovenia    si     svn         95.0      95.0       NaN      94.0   
     spain       es     esp         94.0      98.0      94.0      96.0   
     netherlands nl     nld         94.0      94.0      93.0      94.0   

                                WHS4_544  ROTAC  PCV3  WHS4_543  \
year country     alpha2 alpha3                                    
2004 austria     at     aut         83.0    NaN   NaN       NaN   
     malta       mt     mlt         89.0    NaN   NaN       NaN   
     ireland     ie     irl         89.0    NaN   NaN      30.0   
     netherlands nl     nld         98.0    NaN   NaN       NaN   
     france      fr     fra         99.0    NaN   NaN      85.0   
...                                  ...    ...   ...       ...   
2020 france      fr     fra         96.0    NaN  92.0       NaN   
     cyprus      cy     cyp         96.0    NaN  81.0       NaN   
     slovenia    si     svn         95.0    NaN  70.0       NaN   
     spain       es     esp         94.0    NaN  94.0       NaN   
     netherlands nl     nld         94.0    NaN  93.0       NaN   

                                infant_mortality  education_level  
year country     alpha2 alpha3                                     
2004 austria     at     aut                 4.21             72.5  
     malta       mt     mlt                 6.20             24.0  
     ireland     ie     irl                 4.70             58.6  
     netherlands nl     nld                 4.62             64.2  
     france      fr     fra                 3.44             59.3  
...                                          ...              ...  
2020 france      fr     fra                 3.45             75.1  
     cyprus      cy     cyp                 2.25             75.9  
     slovenia    si     svn                 1.76             83.7  
     spain       es     esp                 2.71             58.0  
     netherlands nl     nld                 3.58             73.2  

[562 rows x 10 columns]

## 3 Visualization

## 4 Immunization rate prediction model

### Implementing Logistic Regression

In [16]:
#Correlation
corr_data = characteristics_data.drop(labels='year_m', axis=1).copy()
corr_data = corr_data[corr_data['IndicatorCode'] != 'PCV3']

display(corr_data.corr())


C:\Users\bamio\AppData\Local\Temp\ipykernel_5536\1191807028.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  display(corr_data.corr())


,infant_mortality,education_level,recommended,mandatory,funded
infant_mortality,1.000000,-0.852058,-0.797942,NaN,-0.797942
education_level,-0.852058,1.000000,0.734155,NaN,0.734155
recommended,-0.797942,0.734155,1.000000,NaN,1.000000
mandatory,NaN,NaN,NaN,NaN,NaN
funded,-0.797942,0.734155,1.000000,NaN,1.000000


In [17]:
from sklearn.linear_model import LogisticRegression
X = corr_data.drop(labels=['immunization', 'IndicatorCode'], axis=1).to_numpy()
y = corr_data['immunization'].fillna(0, inplace=False).to_numpy()

training_X = X[2:]
testing_X = X[0:2]
training_y = y[2:]
testing_y = y[0:2]

model = LogisticRegression(random_state=0).fit(training_X, training_y)
print(f"True immunization: {testing_y}")
print(model.predict(testing_X))


True immunization: [61. 96.]
[85. 85.]


C:\Users\bamio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
